# Industrial demand (historic)

This notebook is to generate historic fluctuations of natural gas consumption by industries, which can further be used as input for industrial fluctuations of hydrogen consumption. 

Note: current historic demand is targeted for 2022 fluctuations, this can further be improved to include other years. 

In [1]:
import os
import sys
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)
os.getcwd()

'C:\\Users\\tatya\\OneDrive - University of Edinburgh\\01 Dissertation\\01 PyPSA\\PyPSA-GB-H2\\notebooks'

In [2]:
import pandas as pd
import numpy as np

In [3]:
# read data for 2022 extracted from National Grid: https://mip-prd-web.azurewebsites.net/DataItemExplorer
xls = pd.ExcelFile('..\data\loads\historic-industrial\industrial-historic-daily-input.xlsx')
df = pd.read_excel(xls, '2022', header=0)

In [4]:
# extract daily industry consumption
df['name'] = df['Adjusted Date']
df['industrial'] = df['Daily, MWh/day']
df = df.filter(['name','industrial'], axis=1)
df = df.set_index(['name'])

In [5]:
# split to equal hourly consumption
df = df.to_period().resample('H').ffill()
df = df / 24

In [6]:
# add total -> share of total
df.loc['total',:] = df.sum(axis=0, numeric_only=True)
df['industrial, share'] = df['industrial'].div(df.iloc[8760]['industrial']).round(100)
df = df.filter(['name','industrial, share'], axis=1)
df = df.drop('total', axis=0)

In [7]:
# print csv
df.to_csv('..\data\loads\historic-industrial\industrial-historic-output.csv', index=True)